In [ ]:
#Checking for NumPy and OpenCV to be installed
#!pip install numpy
#!pip install opencv-python
#!pip install pycoco
#!pip install tensorflow

In [1]:
#Adding paths to directory
import sys
sys.path.insert(0, 'C:/Users/Noel/Untitled Folder/Prophesee Data Set/src/io')
sys.path.insert(0, 'C:/Users/Noel/Untitled Folder/Prophesee Data Set/src/metrics')
sys.path.insert(0, 'C:/Users/Noel/Untitled Folder/Prophesee Data Set/src/visualize')  
sys.path.insert(0, 'C:/Users/Noel/Box/Deep Learning/Prophesee/Prophesee Data Set')  
sys.path.insert(0, 'E:/Prophesee Datasets/detection_dataset_duration_60s_ratio_1.0/train')

from pathlib import Path
import glob
import tensorflow as tf
import numpy as np
import random
import cv2

#Importing the four folders IO, Metrics, Visualize, Dataset_Visualization
import ioio as ioio
import metrics as metrics
import visualize as visualize
import dataset_visualization

#Importing Functions from IO
from ioio import psee_loader, dat_events_tools, npy_events_tools, box_loading, box_filtering
from ioio.psee_loader import PSEELoader
from ioio.dat_events_tools import load_td_data, _dat_transfer, stream_td_data, count_events, parse_header, write_header, write_event_buffer
from ioio.npy_events_tools import stream_td_data, parse_header
from ioio.box_loading import reformat_boxes
from ioio.box_filtering import filter_boxes

#Importing Functions from Metrics
from metrics import coco_eval
from metrics.coco_eval import evaluate_detection, _match_times, _coco_eval, _to_coco_format

#Importing Functions from Visualize
from visualize import vis_utils
from visualize.vis_utils import make_binary_histo, draw_bboxes

#Importing Functions from Dataset_Visualization
from dataset_visualization import play_files_parallel, parse_args


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.test.is_gpu_available()

Num GPUs Available:  1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
#Importing libraries for Neural Network from Keras
from keras import models
from keras import layers

from tensorflow.keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Reshape, Dense, Conv2D, ConvLSTM2D, BatchNormalization

In [4]:
#Run this block of code to see Event Camera Data
'''
#Starting to Visualize Event Camera Data
data_folder = Path("E:/Prophesee Datasets/detection_dataset_duration_60s_ratio_1.0/train")
td_files = glob.glob(str(data_folder / "*.dat"))
play_files_parallel(td_files)
'''

'\n#Starting to Visualize Event Camera Data\ndata_folder = Path("E:/Prophesee Datasets/detection_dataset_duration_60s_ratio_1.0/train")\ntd_files = glob.glob(str(data_folder / "*.dat"))\nplay_files_parallel(td_files)\n'

In [5]:
class_names = ['pedestrians', 'two-wheelers', 'cars']
class_names = np.array(class_names)

Opening a File

In [6]:
# open a file
video = PSEELoader("E:Prophesee Datasets/detection_dataset_duration_60s_ratio_1.0/train/17-04-10_17-59-01_500000_60500000_td.dat")
  
print(video)  # show some metadata
video.event_count()  # number of events in the file
video.total_time()  # duration of the file in mus

PSEELoader:
-----------
Event Type: Event2D
Event Size: 8 bytes
Event Count: 22684222
Duration: 59.999993999999994 s 
-----------



59999994

In [7]:
# to iterate through a dataset you can use :
while not video.done:
    # load events and boxes from all files
    events = video.load_delta_t(10000)
    # do something with the events
    
print(events.shape)
print(events)

(3441,)
[(59990025, 295, 155, 0) (59990025, 295, 155, 0) (59990026, 280, 203, 0)
 ... (59999988, 257, 186, 1) (59999988, 257, 186, 1)
 (59999994,  64, 238, 1)]


In [8]:
#train_labels = np.load("E:Prophesee Datasets/detection_dataset_duration_60s_ratio_1.0/train/17-04-10_17-59-01_500000_60500000_bbox.npy")

#train_labels = np.expand_dims(train_labels, axis=1)
#train_labels = train_labels.reshape(283, 1, 1, 1)
#train_labels = np.tile(train_labels, (1, 64, 64, 1))

#print(train_labels.shape)

In [13]:
# Load the data from the .npy file
train_labels = np.load("E:Prophesee Datasets/detection_dataset_duration_60s_ratio_1.0/train/17-04-10_17-59-01_500000_60500000_bbox.npy")

train_labels = train_labels.reshape(-1)
# Extract the bounding boxes
#bounding_boxes_train = train_labels[:, :4] # Select the first 4 columns (x1, y1, x2, y2)
#train_labels = train_labels[:, 4] # Select the last column (label)
print(train_labels.shape)

(283,)


In [14]:
#val_dataset_folder = Path("E:/Prophesee Datasets/detection_dataset_duration_60s_ratio_1.0/val")
#val_dataset = glob.glob(str(val_dataset_folder / "*.dat"))

val_dataset = PSEELoader("E:Prophesee Datasets/detection_dataset_duration_60s_ratio_1.0/val/17-03-30_12-42-34_500000_60500000_td.dat")

print(val_dataset)

PSEELoader:
-----------
Event Type: Event2D
Event Size: 8 bytes
Event Count: 47892451
Duration: 59.999998999999995 s 
-----------



In [15]:
# to iterate through a dataset you can use :
while not val_dataset.done:
    # load events and boxes from all files
    val_events = val_dataset.load_delta_t(10000)
    # do something with the events
    
print(val_events.shape)
print(val_events)

(19982,)
[(59990000, 32,  73, 0) (59990001,  8,  77, 0) (59990001,  1,  70, 0) ...
 (59999998, 56, 129, 1) (59999999, 43, 133, 1) (59999999, 68, 157, 1)]


In [16]:
# Load the data from the .npy file
data = np.load("E:Prophesee Datasets/detection_dataset_duration_60s_ratio_1.0/val/17-03-30_12-42-34_500000_60500000_bbox.npy")
data = data.reshape(-1,5)
# Extract the bounding boxes
bounding_boxes = data[:, :4] # Select the first 4 columns (x1, y1, x2, y2)
val_targets = data[:, 4] # Select the last column (label)
print(val_targets.shape)

(29,)


In [17]:
def preprocess_event_stream(event_stream, time_window=10, crop_size=128):
    # compute image frames from event stream
    num_events = event_stream.shape[0]
    image_frames = []
    current_frame = np.zeros((crop_size, crop_size), dtype=np.float32)
    current_time = 0
    for i in range(num_events):
        x, y, ts, p = event_stream[i]
        dt = ts - current_time
        if dt >= time_window:
            image_frames.append(current_frame)
            current_frame = np.zeros((crop_size, crop_size), dtype=np.float32)
            current_time = ts
        else:
            dx = int(round(x - crop_size / 2))
            dy = int(round(y - crop_size / 2))
            if dx >= 0 and dx < crop_size and dy >= 0 and dy < crop_size:
                current_frame[dy, dx] += p * dt / time_window

    # apply data augmentation
    num_frames = len(image_frames)
    augmented_frames = []
    for i in range(num_frames):
        # random crop
        x = np.random.randint(0, crop_size - 64)
        y = np.random.randint(0, crop_size - 64)
        cropped_frame = image_frames[i][y:y+64, x:x+64]

        # horizontal flip
        if np.random.rand() < 0.5:
            cropped_frame = np.fliplr(cropped_frame)

        # brightness and contrast adjustment
        alpha = np.random.uniform(0.5, 1.5)
        beta = np.random.uniform(-0.5, 0.5)
        augmented_frame = cv2.convertScaleAbs(cropped_frame, alpha=alpha, beta=beta)

        # Gaussian noise
        noise = np.random.normal(scale=0.1, size=(64, 64))
        noisy_frame = np.clip(augmented_frame.astype(np.float32) + noise, 0, 255).astype(np.uint8)

        augmented_frames.append(noisy_frame)

    # stack frames into tensor
    preprocessed_data = np.stack(augmented_frames, axis=0)
    preprocessed_data = np.expand_dims(preprocessed_data, axis=-1)
    preprocessed_data = preprocessed_data.astype(np.float32) / 255.0

    return preprocessed_data

In [27]:
train_data = preprocess_event_stream(events)

print(train_data.shape)

#print(train_data)

C:\Users\Noel\AppData\Local\Temp\ipykernel_14932\752622027.py:9: RuntimeWarning: overflow encountered in ushort_scalars
  dt = ts - current_time


(1883, 64, 64, 1)
1883


In [19]:
val_data = preprocess_event_stream(val_events)

print(val_data.shape)

C:\Users\Noel\AppData\Local\Temp\ipykernel_14932\752622027.py:9: RuntimeWarning: overflow encountered in ushort_scalars
  dt = ts - current_time


(7970, 64, 64, 1)


In [20]:
def create_event_detector(input_shape=(64, 64, 1)):
    # Input layer
    inputs = tf.keras.layers.Input(shape=input_shape)
    
    # Feature extraction
    x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    features = tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(x)
    
    # Event-attention fusion
    attention = tf.keras.layers.Dense(256, activation='sigmoid')(features)
    fusion = features * attention
    
    # Object detection
    x = tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(fusion)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    
    # Output layer
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
    # Create the model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    return model

In [21]:
model = create_event_detector()
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [24]:
# Train the model
model.fit(train_data, train_labels, batch_size = 32, epochs=10, validation_data=(val_data, val_targets))


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported numpy data type).

In [23]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 32, 32, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________